# ĐỒ ÁN CUỐI KỲ
## Môn học: Toán ứng dụng và thống kê
## Lớp: 21_2
## Họ và tên: Nguyễn Song Toàn
## MSSV: 21120150
## Đồ án số 2

<hr>

### Các hàm hỗ trợ tính toán ma trận dùng cho câu a và b

In [1]:
#Tạo ra ma trận đơn vị kích thước n x n
def create_identity_matrix(n):
    matrix = [[0 for j in range(n)] for i in range(n)]
    for i in range(n):
        matrix[i][i] = 1
    return matrix

#Nhân ma trận với vector
def multiply_matrix_vector(matrix, vector):
    result = [0] * len(matrix)
    for i in range(len(matrix)):
        for j in range(len(vector)):
            result[i] += matrix[i][j] * vector[j]
    return result

#Nhân 2 ma trận
def multiply_two_matrixes(A, B):
    rowsA = len(A)
    colsA = len(A[0])
    rowsB = len(B)
    colsB = len(B[0])
    
    if colsA != rowsB:
        raise ValueError("Số cột của ma trận A phải bằng số dòng của ma trận B")

    result = [[0 for _ in range(colsB)] for _ in range(rowsA)]

    for i in range(rowsA):
        for j in range(colsB):
            for k in range(colsA):
                result[i][j] += A[i][k] * B[k][j]
    return result

#Chuyển vị ma trận A
def transpose_matrix(A):
    m = len(A)
    n = len(A[0])
    # Tạo ma trận B với kích thước (n x m)
    B = [[0 for j in range(m)] for i in range(n)]
    
    # Duyệt qua các phần tử của ma trận A và gán giá trị tại vị trí (i, j) của A bằng giá trị tại vị trí (j, i) của B
    for i in range(m):
        for j in range(n):
            B[j][i] = A[i][j]
    return B

#Hoán vị 2 dòng r1 và r2 của ma trận matrix
def swap_two_rows(matrix, r1, r2):
    matrix[r1], matrix[r2] = matrix[r2], matrix[r1]
    return matrix

#Tìm ma trận nghịch đảo của ma trận A
def inverse(A):
    rows = len(A)
    cols = len(A[0])

    #Tạo ma trận (A|I)
    identityMatrix = []
    for i in range(rows):
        row = [0] * rows
        row[i] = 1
        identityMatrix.append(row)

    matrix = []
    for i in range(rows):
        row = A[i] + identityMatrix[i]
        matrix.append(row)

    for i in range(rows):
        non_zero_idx = -1
        for j in range(i, rows):
            if matrix[j][i] != 0:
                non_zero_idx = j
                break

        if non_zero_idx == -1:
            print ('Ma trận không khả nghịch')
            return None

        # Hoán vị dòng
        swap_two_rows(matrix, non_zero_idx, i)

        # Bước 2: Với số hạng ở dòng thứ i là 𝑎 ≠ 0, nhân dòng i với  1/𝑎 để nhận được số 1 (nằm trên đường chéo của A).
        non_zero_val = matrix[i][i]
        for j in range(i, cols + rows):
            matrix[i][j] /= non_zero_val

        # Bước 3: Cộng một bội số thích hợp của dòng i với các dòng khác dòng i để biến các số hạng trên cột i về số 0 (trừ số hạng nằm ở dòng i).
        for j in range(rows):
            if j != i:
                multiples_num = matrix[j][i]
                for k in range(i, cols * 2):
                    matrix[j][k] -= multiples_num * matrix[i][k]

    # Lấy ma trận nghịch đảo A^{-1}
    inverse_matrix = []
    for i in range(rows):
        row = matrix[i][cols:]
        inverse_matrix.append(row)

    return inverse_matrix

#Trừ 2 vector A - B
def subtract_two_vectors(A, B):
    n = len(A)
    m = len(B)
    if (n != m):
        raise ValueError("Lỗi: Kích thước không hợp lệ!")
    result = [0] * n
    for i in range(n):
        result[i] = A[i] - B[i]
    return result

<hr>

## a. Sử dụng bài toán data fitting trong việc xây dựng mô hình đánh giá giá nhà từ các yếu tố tác động từ dữ liệu được cung cấp.

> Mô hình em sử dụng trong câu a này là mô hình **$ln(Y) = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + \theta_{3} x_{3} + \theta_{4} x_{4}$**                        
> + $x_{1}$ đại diện cho đặc trưng $sqft / 100$             
> + $x_{2}$ đại diện cho đặc trưng Bedrooms                
> + $x_{3}$ đại diện cho đặc trưng Baths
> + $x_{4}$ đại diện cho đặc trưng Age

#### Đọc dữ liệu từ file br2.xlsx

In [2]:
import pandas as pd
file = 'br2.xlsx'
xl = pd.ExcelFile(file)
data = xl.parse('Data')

#### Tạo vector y chứa giá trị ln(price) và ma trận A bao gồm cột chứa toàn số 1 và các đặc trưng còn lại

In [3]:
import math
def create_y_A(data):
    y = []
    for i in range(len(data)):
        y.append(math.log(data.iloc[i, 0]))

    A = []
    for i in range(len(data)):
        row = []
        row.append(1)
        for j in range(1, 5):
            if j == 1:
                row.append(data.iloc[i, j] / 100)
            else:
                row.append(data.iloc[i, j])
        A.append(row)
    return y, A

#### Xây dựng vector các tham số của mô hình
Nghiệm của phương trình hồi quy tuyến tính Ax = y được tính như sau:
$$\hat{x} = (A^{T}A)^{-1}A^{T}y$$

In [4]:
def vectorTheta(A, y):
    temp = []
    temp = multiply_two_matrixes(transpose_matrix(A), A)
    temp = inverse(temp)
    temp1 = []
    temp1 = multiply_matrix_vector(transpose_matrix(A), y)

    Theta = multiply_matrix_vector(temp, temp1)
    return Theta

#### Xuất mô hình khi có danh sách biến và các tham số

In [5]:
def getModel_a(Theta, x):
    if len(Theta) != len(x) + 1:
        raise ValueError("Lỗi: Kích thước không hợp lệ!")

    model = "Y* = lnY = %.5f "%(Theta[0])
    n = len(x)
    for i in range(1, n + 1):
        if Theta[i]==0: continue
        model += " - %0.5f * %s"%(abs(Theta[i]), x[i - 1]) if Theta[i] < 0 else " + %0.5f * %s"%(Theta[i], x[i - 1])
    
    return model

def exportModel_a(A, y, n):
    x = []
    for i in range(1, n + 1):
        x.append("x%d"%(i))
    Theta = vectorTheta(A, y)
    model = getModel_a(Theta, x)
    return model

#### Tạo ra vector $\hat(Y)$ từ ma trận A và vector Theta

In [6]:
def vectorYhat(A, Theta):
    result = []
    for i in range(len(A)):
        s = 0
        for j in range(len(Theta)):
            s += A[i][j] * Theta[j]
        result.append(s)
    return result

#### Tính chuẩn vector phần dư của mô hình

In [7]:
def get_error(y, A):
    Theta = vectorTheta(A, y)
    y_hat = vectorYhat(A, Theta)
    error = 0
    for i in range(len(y)):
        error += (y_hat[i] - y[i]) ** 2;
    return error**0.5

In [26]:
y, A = create_y_A(data)
Theta = vectorTheta(A, y)
print("Mô hình đánh giá giá nhà:")
print(exportModel_a(A, y, 4))
print("Chuẩn vector phần dư là: %.2f"%get_error(y, A))

Mô hình đánh giá giá nhà:
Y* = lnY = 10.91896  + 0.03308 * x1 - 0.05895 * x2 + 0.21457 * x3 - 0.00660 * x4
Chuẩn vector phần dư là: 9.12


<hr>

## b. Sử dụng bài toán data fitting trong xây dựng mô hình đánh giá lương nhân viên từ các yếu tố tác động theo dữ liệu được cung cấp.

> Mô hình em sử dụng trong câu b này là mô hình **$ln(Y) = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + \theta_{3} x_{3}$**                        
> + $x_{1}$ đại diện cho đặc trưng educ             
> + $x_{2}$ đại diện cho đặc trưng exper                
> + $x_{3}$ đại diện cho đặc trưng hrswk

#### Đọc dữ liệu từ file cps4_small.xlsx

In [20]:
file2 = 'cps4_small.xlsx'
xl2 = pd.ExcelFile(file2)
data2 = xl2.parse('Data')

#### Tạo vector y chứa giá trị ln(wage) và ma trận A bao gồm cột chứa toàn số 1 và các đặc trưng còn lại

In [21]:
import math
def create_y_A_b(data):
    y = []
    for i in range(len(data)):
        y.append(math.log(data.iloc[i, 0]))

    A = []
    for i in range(len(data)):
        row = []
        row.append(1)
        for j in range(1, 4):
            row.append(data.iloc[i, j])
        A.append(row)
    return y, A

#### Xuất mô hình khi có danh sách biến và các tham số

In [25]:
y_b, A_b = create_y_A_b(data2)
Theta_b = vectorTheta(A, y)
print("Mô hình đánh giá lương nhân viên:")
print(exportModel_a(A_b, y_b, 3))
print("Chuẩn vector phần dư là: %.2f"%get_error(y_b, A_b))

Mô hình đánh giá lương nhân viên:
Y* = lnY = 1.10054  + 0.09031 * x1 + 0.00578 * x2 + 0.00894 * x3
Chuẩn vector phần dư là: 16.21


<hr>